<a href="https://colab.research.google.com/github/hamzajg/movies-rec-colab/blob/master/MoviesRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
!git clone https://github.com/hamzajg/movies-rec-colab.git

In [63]:
%cd movies-rec-colab/

/content/movies-rec-colab


In [64]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/hamzajg/movies-rec-colab
   63d6b92..4440613  master     -> origin/master
Updating 63d6b92..4440613
Fast-forward
 MovieLens.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)


In [65]:
%cd /content/

/content


In [67]:
import sys
sys.path.insert(0, 'movies-rec-colab')

from MovieLens import MovieLens

ml = MovieLens()
print('Loading Movies Rating..')
data = ml.loadMovieLensLatestSmall()

Loading Movies Rating..


In [69]:
print('Computing movie popularity ranks so we can measure novelty later..')
rankings = ml.getPopularityRanks()

Compute movie popularity ranks so we can measure novelty later..


In [70]:
print('Computing item similarities so we can measure diversity later ...')
fullTrainSet = data.build_full_trainset()

Computing item similarities so we can measure diversity later ...
